# Monroe County Revisited

Originally an exercise at Lighthouse Labs, I wanted to come back to this to expand a little bit.

The point of this exercise is to try data enrichment with data from external APIs and datasets. We are going to take data about car crashes in Monroe County, Indiana from 2003 to 2015 and try to figure out the weather during the accident and how many bars there are in the area. We will work with Foursquare API and Mesonet weather data

We will try to find correlations between the severity of crash and weather/number of bars in the area. To indicate the severity of a crash, we will use column `Injury Type`.

## Data

The data for this exercise can be found [here](https://drive.google.com/file/d/1_KF9oIJV8cB8i3ngA4JPOLWIE_ETE6CJ/view?usp=sharing).


In [1]:
import pandas as pd
import requests as re
import numpy as np
import os
from IPython.display import JSON
import time
from geopy import distance
#import json

In [15]:
accidents_df = pd.read_csv("monroe-county-crash-data2003-to-2015.csv", encoding="unicode_escape")

# preparing data
accidents_df.dropna(subset=['Latitude', 'Longitude'], inplace=True)
# creation of variable with lon and lat together
accidents_df['ll'] = accidents_df['Latitude'].astype(str) + ',' + accidents_df['Longitude'].astype(str)
# remove 0 lat and lon
accidents_df = accidents_df[accidents_df['ll'] != '0.0,0.0']

# rename columns
accidents_df.columns =[column.replace(" ", "_") for column in accidents_df.columns]
print(accidents_df.shape)
accidents_df.head()

(49005, 13)


,Master_Record_Number,Year,Month,Day,Weekend?,Hour,Collision_Type,Injury_Type,Primary_Factor,Reported_Location,Latitude,Longitude,ll
0,902363382,2015,1,5,Weekday,0.0,2-Car,No injury/unknown,OTHER (DRIVER) - EXPLAIN IN NARRATIVE,1ST & FESS,39.159207,-86.525874,"39.15920668,-86.52587356"
1,902364268,2015,1,6,Weekday,1500.0,2-Car,No injury/unknown,FOLLOWING TOO CLOSELY,2ND & COLLEGE,39.161440,-86.534848,"39.16144,-86.534848"
2,902364412,2015,1,6,Weekend,2300.0,2-Car,Non-incapacitating,DISREGARD SIGNAL/REG SIGN,BASSWOOD & BLOOMFIELD,39.149780,-86.568890,"39.14978027,-86.56889006"
3,902364551,2015,1,7,Weekend,900.0,2-Car,Non-incapacitating,FAILURE TO YIELD RIGHT OF WAY,GATES & JACOBS,39.165655,-86.575956,"39.165655,-86.57595635"
4,902364615,2015,1,7,Weekend,1100.0,2-Car,No injury/unknown,FAILURE TO YIELD RIGHT OF WAY,W 3RD,39.164848,-86.579625,"39.164848,-86.57962482"


In [16]:
# Make date from columns Year, Month, Day
accidents_df['Date'] = pd.to_datetime(accidents_df[['Year', 'Month', 'Day']])

In [17]:
# Impute Weekend? from Date due to missing values
accidents_df['Weekend?'] = accidents_df['Date'].dt.dayofweek >= 5

In [18]:
# drop NaN Hour
accidents_df.dropna(subset=['Hour'], inplace=True)
# drop NaN Collision_Type
accidents_df.dropna(subset=['Collision_Type'], inplace=True)

In [19]:
# format Hour from float to int
accidents_df['Hour'] = accidents_df['Hour'].astype(int)

In [20]:
# make timestamp column from accidents_df Date and Hour columns
accidents_df['Timestamp'] = pd.to_datetime(accidents_df['Date'].astype(str) + accidents_df['Hour'].astype(str).str.zfill(4), format='%Y-%m-%d%H%M')

In [23]:
# selecting only the columns we need
accidents_df = accidents_df[['Timestamp', 'Weekend?', 'Collision_Type', 'Injury_Type', 'Primary_Factor', 'll']]

In [24]:
# show Primary_Factor types
accidents_df['Primary_Factor'].value_counts()

FAILURE TO YIELD RIGHT OF WAY               10287
FOLLOWING TOO CLOSELY                        6741
OTHER (DRIVER) - EXPLAIN IN NARRATIVE        5477
UNSAFE BACKING                               4701
RAN OFF ROAD RIGHT                           2641
DISREGARD SIGNAL/REG SIGN                    2084
SPEED TOO FAST FOR WEATHER CONDITIONS        1757
IMPROPER TURNING                             1699
DRIVER DISTRACTED - EXPLAIN IN NARRATIVE     1505
ANIMAL/OBJECT IN ROADWAY                     1467
UNSAFE SPEED                                 1337
ROADWAY SURFACE CONDITION                    1110
LEFT OF CENTER                                940
IMPROPER LANE USAGE                           893
ALCOHOLIC BEVERAGES                           717
UNSAFE LANE MOVEMENT                          698
OVERCORRECTING/OVERSTEERING                   518
IMPROPER PASSING                              454
OTHER (VEHICLE) - EXPLAIN IN NARRATIVE        419
OTHER (ENVIRONMENTAL) - EXPLAIN IN NARR       345


In [25]:
# Check NaN
accidents_df.isnull().sum()

Timestamp           0
Weekend?            0
Collision_Type      0
Injury_Type         0
Primary_Factor    969
ll                  0
dtype: int64

In [26]:
# hot encode Injury_Type
accidents_df = pd.get_dummies(accidents_df, columns=['Injury_Type'])

In [27]:
accidents_df

,Timestamp,Weekend?,Collision_Type,Primary_Factor,ll,Injury_Type_Fatal,Injury_Type_Incapacitating,Injury_Type_No injury/unknown,Injury_Type_Non-incapacitating
50591,2003-01-01 03:00:00,False,2-Car,FAILURE TO YIELD RIGHT OF WAY,"39.171632,-86.523536",0,0,1,0
50150,2003-01-01 03:00:00,False,2-Car,UNSAFE BACKING,"39.178848,-86.511088",0,0,1,0
51947,2003-01-01 20:00:00,False,2-Car,FAILURE TO YIELD RIGHT OF WAY,"39.163328,-86.525808",0,0,1,0
53009,2003-01-01 11:00:00,False,2-Car,OTHER (DRIVER) - EXPLAIN IN NARRATIVE,"39.12631655,-86.52674475",0,0,1,0
50084,2003-01-01 20:00:00,False,1-Car,RAN OFF ROAD RIGHT,"39.178976,-86.5224",0,0,1,0
...,...,...,...,...,...,...,...,...,...
4016,2015-12-07 03:00:00,False,1-Car,UNSAFE SPEED,"39.171648,-86.50435515",0,0,1,0
4015,2015-12-07 00:00:00,False,2-Car,DISREGARD SIGNAL/REG SIGN,"39.16464,-86.5336",0,0,1,0
4203,2015-12-07 10:00:00,False,2-Car,UNSAFE BACKING,"39.14648987,-86.53246164",0,0,1,0
4212,2015-12-07 20:00:00,False,1-Car,RAN OFF ROAD RIGHT,"39.15473522,-86.5731008",0,1,0,0


# Foursquare API

Foursquare API documentation is [here](https://developer.foursquare.com/)

1. Start a foursquare application and get your keys.
2. For each crash, create the function **get_venues** that will pull bars in the radius of 5km around the crash

#### example
`get_venues('48.146394, 17.107969')`

3. Find a relationship (if there is any) between number of bars in the area and severity of the crash.

HINTs: 
- check out python package "foursquare" (no need to send HTTP requests directly with library `requests`)
- **categoryId** for bars and nightlife needs to be found in the [foursquare API documentation](https://developer.foursquare.com/docs/api-reference/venues/search/)

In [28]:
#set the keys
foursquare_id = os.environ["FS_CLIENT_ID"]
foursquare_secret = os.environ["FS_CL_SECRET"]
foursquare_api = os.environ["FS_API_KEY"]

In [29]:
#initialize FS API
headers = {

    "Accept": "application/json",

    "Authorization": foursquare_api

}
url="https://api.foursquare.com/v3/places/search"
radius = "&radius=10000"
limit = "&limit=50"
citycenter="39.1676747,-86.5314594"

In [30]:
#get venues list given a set of coordinates from FS API
def get_venues(coords):
    stripcoords = coords.replace(" ", "")
    url_keys_append = "?query=bar&ll=" + stripcoords + radius + limit
    requrl = url + url_keys_append
    res = re.request("GET", requrl, headers=headers)
    print(res.status_code)
    dataset = res.json()
    return dataset;


In [31]:
def fs_get_rect(northeast, southwest):
    params = {
		"query": "bar",
  	"ne": northeast,
  	"sw": southwest
	}
    response = re.request("GET", url, params=params, headers=headers)
    if response.status_code != 200:
        print("Error:", response.status_code)
        return None
    else:
        barset = pd.DataFrame(columns=['name', 'lat', 'lng'])
        data = response.json()
        bars = data['results']
        for bar in bars:
            barset = barset.append({
                'name': bar['name'],
                'lat': bar['geocodes']['main']['latitude'],
                'lng': bar['geocodes']['main']['longitude']
            }, ignore_index=True)
        return barset

In [32]:
# test fs_get_rect
resulttest = fs_get_rect("39.2525,-86.3656", "39.2400,-86.4656")

In [33]:
resulttest

,name,lat,lng
0,Butler Winery & Vineyard,39.241777,-86.44664


In [34]:
# get all bar locations from FS API using rectangular boundary
def get_venues_loop(start_lat, start_lon, end_lat, end_lon):
	lat_point = start_lat
	lon_point = start_lon
	lat_step = 0.0292
	lon_step = 0.0155
	all_bars = pd.DataFrame(columns=['name', 'lat', 'lng'])

	#iterate through the rectangle
	while lat_point > end_lat:
		while lon_point > end_lon:
			# get the response from the FS API
			response = fs_get_rect(str(lat_point)+","+str(lon_point), str(lat_point - lat_step)+","+str(lon_point - lon_step))
			if response is not None:
				all_bars = all_bars.append(response, ignore_index=True)
			# sleep for 5 second to avoid rate limiting
			lon_point -= lon_step
			time.sleep(5)
		lon_point = start_lon
		lat_point -= lat_step
	return all_bars
		

In [35]:
# starting coordinates is roughly top right corner of where the accidents are
# end point is bottom left corner
start_lat = 39.3525
end_lat = 39.0425
start_lon = -86.3656
end_lon = -86.7067

In [34]:
bars = get_venues_loop(start_lat, start_lon, end_lat, end_lon)

In [37]:
bars

,name,lat,lng
0,Gosport Tavern,39.349143,-86.663583
1,Barn of Terror,39.304334,-86.487356
2,Mulligan's,39.295986,-86.692026
3,Blue Barn Antiques,39.281427,-86.511780
4,Butler Winery & Vineyard,39.240651,-86.445828
...,...,...,...
100,Workman Pole Barns & Garages,39.068751,-86.695921
101,Fairfax Inn,39.034080,-86.490740
102,The Pointe Bar,39.033185,-86.511288
103,Stonecreek Ranch Barn,39.059434,-86.604144


In [36]:
# remove duplicates
bars.drop_duplicates(inplace=True)

In [38]:
# create ll column from lat and lng
bars['ll'] = bars['lat'].astype(str) + ',' + bars['lng'].astype(str)

In [39]:
bars.to_csv('bars.csv', index=False)

In [36]:
bars = pd.read_csv('bars.csv')

In [37]:
bars


,name,lat,lng,ll
0,Gosport Tavern,39.349143,-86.663583,"39.349143,-86.663583"
1,Barn of Terror,39.304334,-86.487356,"39.304334,-86.487356"
2,Mulligan's,39.295986,-86.692026,"39.295986,-86.692026"
3,Blue Barn Antiques,39.281427,-86.511780,"39.281427,-86.51178"
4,Butler Winery & Vineyard,39.240651,-86.445828,"39.240651,-86.445828"
...,...,...,...,...
99,Workman Pole Barns & Garages,39.068751,-86.695921,"39.068751,-86.695921"
100,Fairfax Inn,39.034080,-86.490740,"39.03408,-86.49074"
101,The Pointe Bar,39.033185,-86.511288,"39.033185,-86.511288"
102,Stonecreek Ranch Barn,39.059434,-86.604144,"39.059434,-86.604144"


In [38]:
accidents_df['closest_bar'] = np.nan
accidents_df['number_of_bars_1km'] = np.nan
accidents_df['number_of_bars_3km'] = np.nan

In [39]:
# for each accident add a column with the closest bar distance and columns with amount of bars in 1km and 3km radius

for acc_index, accident in accidents_df.iterrows():
    closest_bar = 9000
    number_of_bars_3 = 0
    number_of_bars_1 = 0
    for _, bar in bars.iterrows():
        distance_from_bar = distance.distance(accident['ll'], bar['ll']).kilometers
        if distance_from_bar < 1:
            number_of_bars_1 += 1
        if distance_from_bar < 3:
            number_of_bars_3 += 1
        if distance_from_bar < closest_bar:
            closest_bar = distance_from_bar
    accidents_df.loc[acc_index, 'closest_bar'] = closest_bar
    accidents_df.loc[acc_index, 'number_of_bars_1km'] = number_of_bars_1
    accidents_df.loc[acc_index, 'number_of_bars_3km'] = number_of_bars_3

In [40]:
accidents_df

,Timestamp,Weekend?,Collision_Type,Primary_Factor,ll,Injury_Type_Fatal,Injury_Type_Incapacitating,Injury_Type_No injury/unknown,Injury_Type_Non-incapacitating,closest_bar,number_of_bars_1km,number_of_bars_3km
50591,2003-01-01 03:00:00,False,2-Car,FAILURE TO YIELD RIGHT OF WAY,"39.171632,-86.523536",0,0,1,0,0.678816,4.0,40.0
50150,2003-01-01 03:00:00,False,2-Car,UNSAFE BACKING,"39.178848,-86.511088",0,0,1,0,0.790025,2.0,32.0
51947,2003-01-01 20:00:00,False,2-Car,FAILURE TO YIELD RIGHT OF WAY,"39.163328,-86.525808",0,0,1,0,0.453060,12.0,45.0
53009,2003-01-01 11:00:00,False,2-Car,OTHER (DRIVER) - EXPLAIN IN NARRATIVE,"39.12631655,-86.52674475",0,0,1,0,0.440370,5.0,12.0
50084,2003-01-01 20:00:00,False,1-Car,RAN OFF ROAD RIGHT,"39.178976,-86.5224",0,0,1,0,0.196163,2.0,34.0
...,...,...,...,...,...,...,...,...,...,...,...,...
4016,2015-12-07 03:00:00,False,1-Car,UNSAFE SPEED,"39.171648,-86.50435515",0,0,1,0,0.108901,3.0,37.0
4015,2015-12-07 00:00:00,False,2-Car,DISREGARD SIGNAL/REG SIGN,"39.16464,-86.5336",0,0,1,0,0.108533,18.0,36.0
4203,2015-12-07 10:00:00,False,2-Car,UNSAFE BACKING,"39.14648987,-86.53246164",0,0,1,0,1.011308,0.0,35.0
4212,2015-12-07 20:00:00,False,1-Car,RAN OFF ROAD RIGHT,"39.15473522,-86.5731008",0,1,0,0,0.987490,1.0,20.0


### Mesonet ASOS weather data

https://mesonet.agron.iastate.edu/ASOS

In [42]:
weather_df = pd.read_csv("BMG.csv")

In [43]:
weather_df.rename(columns = {'valid': 'Time', 'sknt':'Wind_Speed', 'p01i':'Precipitation',
                              'vsby':'Visibility', 'gust':'Wind_Gust', 'wxcodes': 'Weather Codes', 'ice_accretion_1hr': 'Ice_Accretion'}, inplace = True)

In [44]:
# make timestamp out of 'Time' in weather_df
weather_df['Time'] = pd.to_datetime(weather_df['Time'])

Weather phenomena:
RA Rain SN Snow SG Snow Grains
DZ Drizzle IC Ice Crystals PL Ice pellets (sleet)
GS Small hail GR Hail UP Unknown precipitation
Obscurations to visibility:
BR Mist (>=5/8 mi) FG Fog (< 5/8 mi)
FU Smoke VA Volcanic Ash
SA Sand HZ Haze
PY Spray DU Widespread Dust
Other:
SQ Squall (strong wind) SS Sandstorm
DS Duststorm PO Dust/sand whirls
FC Funnel Cloud FC+ Tornado/waterspout
Qualifiers (for RA, DZ, SN, PL):
- Light
(No sign) Moderate
+ Heavy
VC Vicinity
Examples:
+RA Heavy Rain
-DZ Light Drizzle
SN Moderate Snow
VCTS Thunderstorm in the vicinity (5-10 mi from observation)
Other Descriptors:
MI Shallow BC Patches PR Partial
TS Thunderstorm BL Blowing SH Showers
DR Drifting FZ Freezing
Examples:
BCFG Patchy fog
+TSRA Thunderstorm with heavy rain
BLSN Blowing snow
SHRA Moderate rain showers
TSRAGR Thunderstorm with moderate rain and hail

In [45]:
weather_df

,station,Time,Wind_Speed,Precipitation,Visibility,Wind_Gust,Weather Codes,Ice_Accretion
0,BMG,2003-01-01 00:30:00,10.00,M,9.00,M,M,M
1,BMG,2003-01-01 00:53:00,11.00,T,10.00,17.00,-RA,M
2,BMG,2003-01-01 01:20:00,10.00,0.01,4.00,M,-RA BR,M
3,BMG,2003-01-01 01:53:00,11.00,0.03,5.00,M,-RA BR,M
4,BMG,2003-01-01 02:53:00,10.00,0.04,4.00,M,-RA BR,M
...,...,...,...,...,...,...,...,...
152401,BMG,2015-12-07 22:53:00,0.00,0.00,5.00,M,BR,0.01
152402,BMG,2015-12-07 23:05:00,3.00,0.00,5.00,M,BR,M
152403,BMG,2015-12-07 23:53:00,3.00,0.00,2.50,M,BR,M
152404,BMG,2015-12-07 23:55:00,3.00,0.00,1.25,M,BR,M


In [47]:
# append weather data to each accident. match closest timestamp
for acc_index, accident in accidents_df.iterrows():
    # get the closes timestamp in weather_df
    closest_time = weather_df.iloc[(weather_df['Time']-accident['Timestamp']).abs().argsort()[:1]]

    # append weather data to accident
    accidents_df.loc[acc_index, 'Wind_Speed'] = closest_time['Wind_Speed'].values[0]
    accidents_df.loc[acc_index, 'Precipitation'] = closest_time['Precipitation'].values[0]
    accidents_df.loc[acc_index, 'Visibility'] = closest_time['Visibility'].values[0]
    accidents_df.loc[acc_index, 'Wind_Gust'] = closest_time['Wind_Gust'].values[0]
    accidents_df.loc[acc_index, 'Weather Codes'] = closest_time['Weather Codes'].values[0]
    accidents_df.loc[acc_index, 'Ice_Accretion'] = closest_time['Ice_Accretion'].values[0]
    
    

In [48]:
accidents_df

,Timestamp,Weekend?,Collision_Type,Primary_Factor,ll,Injury_Type_Fatal,Injury_Type_Incapacitating,Injury_Type_No injury/unknown,Injury_Type_Non-incapacitating,closest_bar,number_of_bars_1km,number_of_bars_3km,Wind_Speed,Precipitation,Visibility,Wind_Gust,Weather Codes,Ice_Accretion
50591,2003-01-01 03:00:00,False,2-Car,FAILURE TO YIELD RIGHT OF WAY,"39.171632,-86.523536",0,0,1,0,0.678816,4.0,40.0,10.00,0.04,4.00,M,-RA BR,M
50150,2003-01-01 03:00:00,False,2-Car,UNSAFE BACKING,"39.178848,-86.511088",0,0,1,0,0.790025,2.0,32.0,10.00,0.04,4.00,M,-RA BR,M
51947,2003-01-01 20:00:00,False,2-Car,FAILURE TO YIELD RIGHT OF WAY,"39.163328,-86.525808",0,0,1,0,0.453060,12.0,45.0,8.00,M,10.00,M,M,M
53009,2003-01-01 11:00:00,False,2-Car,OTHER (DRIVER) - EXPLAIN IN NARRATIVE,"39.12631655,-86.52674475",0,0,1,0,0.440370,5.0,12.0,13.00,0.01,5.00,19.00,-RA BR,M
50084,2003-01-01 20:00:00,False,1-Car,RAN OFF ROAD RIGHT,"39.178976,-86.5224",0,0,1,0,0.196163,2.0,34.0,8.00,M,10.00,M,M,M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4016,2015-12-07 03:00:00,False,1-Car,UNSAFE SPEED,"39.171648,-86.50435515",0,0,1,0,0.108901,3.0,37.0,0.00,0.00,10.00,M,M,M
4015,2015-12-07 00:00:00,False,2-Car,DISREGARD SIGNAL/REG SIGN,"39.16464,-86.5336",0,0,1,0,0.108533,18.0,36.0,4.00,0.00,10.00,M,M,M
4203,2015-12-07 10:00:00,False,2-Car,UNSAFE BACKING,"39.14648987,-86.53246164",0,0,1,0,1.011308,0.0,35.0,0.00,0.00,9.00,M,M,M
4212,2015-12-07 20:00:00,False,1-Car,RAN OFF ROAD RIGHT,"39.15473522,-86.5731008",0,1,0,0,0.987490,1.0,20.0,0.00,0.00,4.00,M,BR,M


In [49]:
accidents_df.to_csv("accidents.csv")

# Data Preparation